In [3]:
import numpy as np
import theano
from theano import tensor as T

In [9]:
num_steps = 5
batch_size = 3
emb_dim = 7

batch_range = T.arange(batch_size, dtype="int32")

In [4]:
stack = theano.shared(np.zeros((num_steps * batch_size, emb_dim), dtype=np.float32), name="stack")
stack_grad = theano.shared(np.zeros_like(stack.get_value()), name="stack_grad")

In [5]:
cursors = T.ones((batch_size,), dtype=np.int32) * -1
queue = T.zeros((batch_size * num_steps,), dtype=np.int32)

In [28]:
Xs = T.tensor3("Xs")
masks = T.imatrix("masks")

In [29]:
def stack_op(t, stack_t, stack_grad_t, push_value, merge_value,
             queue_t, cursors_t, mask_t, batch_size, stack_size, batch_range):
    
    mask2 = T.cast(mask_t.dimshuffle(0, "x"), "float32")
    top_next = mask2 * merge_value + (1 - mask2) * push_value
    stack_next = T.set_subtensor(stack_t[t * batch_size + batch_range], top_next)
    
    # TODO grad_next
    stack_grad_next = stack_grad_t
    
    cursors_next = cursors_t + (mask_t * -1 + (1 - mask_t) * 1)
    queue_next = T.set_subtensor(queue_t[batch_range * stack_size + cursors_next], t)
    
    return stack_next, stack_grad_next, queue_next, cursors_next

In [30]:
def step(t, x_t, mask_t,       # sequences
         queue_t, cursors_t,   # outputs_info
         stack_t, stack_grad_t # non_sequences
        ):
    
    stack1 = stack_t[(t - 1) * batch_size + batch_range]
    
    stack2_ptrs = queue_t[cursors_t - 1 + batch_range * num_steps]
    stack2 = stack_t[stack2_ptrs * batch_size + batch_range].reshape((batch_size, emb_dim))
    
    # Merge op: just add.
    merge_value = stack1 + stack2
    
    stack_next, stack_grad_next, queue_next, cursors_next = stack_op(
        t, stack_t, stack_grad_t, x_t, merge_value, queue_t, cursors_t,
        mask_t, batch_size, num_steps, batch_range)
    
    outputs = (queue_next, cursors_next)
    updates = {stack: stack_next,
               stack_grad: stack_grad_next}
    return outputs, updates

In [31]:
scan_out, scan_updates = theano.scan(
    step, [T.arange(num_steps), Xs, masks],
    outputs_info=[queue, cursors],
    non_sequences=[stack, stack_grad])

In [32]:
f_clear = theano.function([], [], updates={stack: T.zeros_like(stack),
                                           stack_grad: T.zeros_like(stack_grad)})

/Users/jon/Projects/ipython-notebooks/.env/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: The parameter 'updates' of theano.function() expects an OrderedDict, got <type 'dict'>. Using a standard dictionary here results in non-deterministic behavior. You should use an OrderedDict if you are using Python 2.7 (theano.compat.OrderedDict for older python), or use a list of (shared, update) pairs. Do not just convert your dictionary to this type before the call as the conversion will still be non-deterministic.
  from ipykernel import kernelapp as app


In [33]:
f_ff = theano.function([Xs, masks], stack, updates=scan_updates)

<class 'theano.tensor.subtensor.IncSubtensor'>
<class 'theano.tensor.subtensor.IncSubtensor'>
<class 'theano.tensor.subtensor.IncSubtensor'>
<class 'theano.tensor.subtensor.IncSubtensor'>
IncSubtensor1 <class 'theano.tensor.subtensor.AdvancedIncSubtensor1'>
	AdvancedIncSubtensor1{no_inplace,set}(<TensorType(int32, vector)>, <TensorType(int64, scalar)>, Elemwise{add,no_inplace}.0)
	AdvancedIncSubtensor1{inplace,set}.0
IncSubtensor1 <class 'theano.tensor.subtensor.AdvancedIncSubtensor1'>
	AdvancedIncSubtensor1{no_inplace,set}(stack_copy, Elemwise{Composite{((i0 * (i1 + i2)) + i3)}}.0, <TensorType(int64, vector)>)
	AdvancedIncSubtensor1{inplace,set}.0


In [36]:
X_test = np.array([np.arange(batch_size)[:, np.newaxis].repeat(emb_dim, axis=1) + 1
                   for t in range(num_steps)], dtype=np.int32)
mask_test = np.array([[0, 0, 1, 0, 1],
                      [0, 0, 0, 1, 0],
                      [0, 0, 0, 0, 0]], dtype=np.int32).T

In [43]:
f_clear()
f_ff(X_test, mask_test)
stack.get_value()

array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 2.,  2.,  2.,  2.,  2.,  2.,  2.],
       [ 3.,  3.,  3.,  3.,  3.,  3.,  3.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 2.,  2.,  2.,  2.,  2.,  2.,  2.],
       [ 3.,  3.,  3.,  3.,  3.,  3.,  3.],
       [ 2.,  2.,  2.,  2.,  2.,  2.,  2.],
       [ 2.,  2.,  2.,  2.,  2.,  2.,  2.],
       [ 3.,  3.,  3.,  3.,  3.,  3.,  3.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 4.,  4.,  4.,  4.,  4.,  4.,  4.],
       [ 3.,  3.,  3.,  3.,  3.,  3.,  3.],
       [ 3.,  3.,  3.,  3.,  3.,  3.,  3.],
       [ 2.,  2.,  2.,  2.,  2.,  2.,  2.],
       [ 3.,  3.,  3.,  3.,  3.,  3.,  3.]], dtype=float32)